In [1]:
# GOOGLE DRIVE KA PURA TREE BANANE WALA FINAL CODE (Roman Urdu Comments)

from googleapiclient.discovery import build
from google.colab import auth, drive, files
from google.auth import default
from IPython.display import display, HTML
import time

# Step 1: Drive mount karo aur permission lo
drive.mount('/content/drive')
auth.authenticate_user()
creds, _ = default()
service = build('drive', 'v3', credentials=creds)

# File ka naam aur location jahan HTML save hoga
output_file = "/content/drive/MyDrive/Google_Drive_Tree_RomanUrdu.html"
print("Google Drive scan kar raha hun...")

def get_all():
    """Saari files aur folders ek saath laao"""
    items = []
    token = None
    while True:
        result = service.files().list(
            q="trashed = false",
            spaces='drive',
            fields='nextPageToken, files(id,name,mimeType,parents,size)',
            pageSize=1000,
            pageToken=token
        ).execute()

        items.extend(result.get('files', []))
        print(f"Ab tak {len(items)} items mile...", end='\r')
        token = result.get('nextPageToken')
        if not token:
            break
    print(f"\nTotal items mile: {len(items)}")
    return items

# Saari files fetch karo
all_items = get_all()

# ID se jaldi dhundne ke liye dictionary banao
by_id = {item['id']: item for item in all_items}

# Har folder ke andar kon-kon se bachche (children) hain, yeh dikhao
children = {}
for item in all_items:
    for parent_id in item.get('parents', []):
        children.setdefault(parent_id, []).append(item)

# Jo files/folders kisi ke andar nahi hain, woh root hain (My Drive ya Shared Drives)
roots = [item for item in all_items if not item.get('parents') or item['parents'][0] not in by_id]

def sort_kids(kids):
    """Pehle folders, phir files — dono alphabetically"""
    folders = [x for x in kids if x['mimeType'] == 'application/vnd.google-apps.folder']
    files_only = [x for x in kids if x['mimeType'] != 'application/vnd.google-apps.folder']
    folders.sort(key=lambda x: x['name'].lower())
    files_only.sort(key=lambda x: x['name'].lower())
    return folders + files_only

# Har folder ko ek unique ID do taaki collapse/open kar sakein
folder_counter = 0
def new_id():
    global folder_counter
    folder_counter += 1
    return f"f{folder_counter}"

def build(item, prefix="", is_last=True):
    """Tree ki har line yahan banegi"""
    name = item['name']
    file_id = item['id']
    is_folder = item['mimeType'] == 'application/vnd.google-apps.folder'

    # Tree mein └── ya ├── lagao
    connector = "└── " if is_last else "├── "
    indent = prefix + ("    " if is_last else "│   ")

    # File size dikhao (sirf files ke liye)
    size_text = ""
    if not is_folder and 'size' in item:
        size = int(item['size'])
        if size >= 1073741824:
            size_text = f" ({size/1073741824:.2f} GB)"
        elif size >= 1048576:
            size_text = f" ({size/1048576:.1f} MB)"
        elif size > 0:
            size_text = f" ({size//1024} KB)"

    if is_folder:
        folder_html_id = new_id()
        line = f'{prefix}{connector}<span class="fold" onclick="t(\'{folder_html_id}\')">Folder <b>{name}</b></span><br>'
        line += f'<div id="{folder_html_id}" class="sub">'
    else:
        view_link = f"https://drive.google.com/file/d/{file_id}/view"
        download_link = f"https://drive.google.com/uc?id={file_id}&export=download"
        line = f'{prefix}{connector}File <a href="{view_link}" target="_blank" class="fn">{name}</a> ' \
               f'<span class="sz">{size_text}</span> ' \
               f'<a href="{download_link}" download class="dl">[Download]</a><br>'

    # Bachchon ko bhi add karo
    kids = sort_kids(children.get(file_id, []))
    for i, kid in enumerate(kids):
        line += build(kid, indent, i == len(kids)-1)

    if is_folder:
        line += '</div>'
    return line

# Pura tree banao
tree_html = ""
for root in sort_kids(roots):
    if root['id'] == 'root' or root.get('name') == 'My Drive':
        tree_html += '<div class="rootname">My Drive</div>'
    else:
        tree_html += f'<div class="shared">Shared Drive: <b>{root["name"]}</b></div>'
    tree_html += build(root, "", True)

total_items = len(all_items)

# Final HTML (JavaScript mein ${} nahi use kiya taaki Colab mein error na aaye)
html = f'''<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>Google Drive Tree - Roman Urdu Version</title>
<style>
  body{{font-family:Segoe UI,Arial;background:#f0f2f5;padding:20px;margin:0}}
  h1{{color:#1967d2;text-align:center}}
  input{{width:100%;padding:16px;font-size:20px;border:3px solid #1967d2;border-radius:12px;box-sizing:border-box;margin:15px 0}}
  .info{{text-align:center;margin:15px;font-size:18px;color:#555}}
  .tree{{background:#fff;padding:30px;border-radius:16px;box-shadow:0 10px 30px rgba(0,0,0,0.15);font-family:Consolas,monospace;font-size:15px;line-height:1.8}}
  .fold{{cursor:pointer;color:#1967d2;font-weight:bold}}
  .fold:hover{{text-decoration:underline}}
  .fn{{color:#1a0dab;text-decoration:none}}
  .fn:hover{{text-decoration:underline}}
  .sz{{color:#777;font-size:0.9em}}
  .dl{{margin-left:15px;color:#c5221f;font-weight:bold}}
  .sub{{padding-left:30px;display:none}}
  .rootname,.shared{{font-size:20px;font-weight:bold;color:#1967d2;margin:20px 0 10px}}
  .hl{{background:#ffeb3b !important;padding:2px 6px;border-radius:4px}}
</style>
</head>
<body>
<h1>Google Drive - Pura Tree (Collapsible)</h1>
<input type="text" placeholder="File ka naam search karo..." autofocus>
<div class="info">Total items: {total_items}</div>
<div class="tree">{tree_html}</div>

<script>
// Folder open/close karne ka function
function t(id){{
  var x = document.getElementById(id);
  if (x.style.display === "none" || x.style.display === "none") {{
    x.style.display = "block";
  }} else {{
    x.style.display = "none";
  }}
}}

var input = document.querySelector('input');
var info = document.querySelector('.info');
var found = 0;

input.oninput = function() {{
  var term = input.value.trim().toLowerCase();
  var names = document.querySelectorAll('.fn');
  found = 0;

  names.forEach(function(el) {{
    var text = el.textContent.toLowerCase();
    if (term && text.includes(term)) {{
      el.classList.add('hl');
      found++;
      // Parent folders bhi khol do taaki result dikhe
      var p = el.parentElement;
      while (p) {{
        if (p.classList && p.classList.contains('sub')) p.style.display = 'block';
        p = p.parentElement;
      }}
    }} else {{
      el.classList.remove('hl');
    }}
  }});

  if (term) {{
    info.textContent = "Found " + found + " result" + (found == 1 ? "" : "s") + ' — "' + input.value + '"';
    if (found > 0) document.querySelector('.hl').scrollIntoView({{behavior:'smooth', block:'center'}});
  }} else {{
    info.textContent = "Total items: {total_items}";
  }}
}};
</script>
</body>
</html>'''

# File save karo aur download karwao
with open(output_file, "w", encoding="utf-8") as f:
    f.write(html)

print("\nHo gaya bhai! Sab kuch perfect!")
print("File save hui →", output_file)
files.download(output_file)
display(HTML(html))

Mounted at /content/drive


Google Drive scan kar raha hun...
Ab tak 5735 items mile...
Total items mile: 5735

Ho gaya bhai! Sab kuch perfect!
File save hui → /content/drive/MyDrive/Google_Drive_Tree_RomanUrdu.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>